In [14]:
import pandas as pd
import requests
import json
import geohash2

# 직방의 원룸 매물정보 수집
- 절차
    - 동이름 > 위도, 경도
    - 위도, 경도 > geohash(영역) 변환
    - geohash(영역) > 매물아이디
    - 매물아이디 > 매물정보


## 1. 동이름 > 위도, 경도

In [100]:
address = '강남구 역삼동'
serviceType = '원룸'
url = f'https://apis.zigbang.com/v2/search?leaseYn=N&q={address}&serviceType={serviceType}'

In [101]:
response = requests.get(url)
response

<Response [200]>

In [102]:
data = response.json()['items'][0]
lat, lng = data['lat'], data['lng']
lat, lng

(37.495365142822266, 127.03306579589844)

## 2. 위도, 경도 > geohash

In [103]:
# precision이 커질수록 영역이 작아짐
geohash = geohash2.encode(lat, lng, precision=5)
geohash

'wydm6'

## 3. geohash > 매물 아이디

In [116]:
item_list_url = 'https://apis.zigbang.com/v2/items?domain=zigbang'\
f'&geohash={geohash}&needHasNoFiltered=true&new_villa=true&sales_type_in=전세|월세&zoom=16'

item_list_url = 'https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang'\
f'&geohash={geohash}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸'

print(item_list_url)

https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang&geohash=wydm6&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸


In [117]:
response = requests.get(item_list_url)
response

<Response [200]>

In [118]:
items = response.json()['items']

In [119]:
len(items)

2336

In [120]:
ids = [item['item_id'] for item in items]
ids[:5]

[32819635, 32838798, 32872608, 32838700, 32767020]

## 4. 매물 아이디 > 매물 정보

In [127]:
get_item_list = 'https://apis.zigbang.com/v2/items/list'
params = {"domain": "zigbang",
          "withCoalition": 'true',
          "item_ids": ids[:900]
          }

response = requests.post(get_item_list, data=params)
response

<Response [200]>

In [128]:
items = response.json()['items']
df = pd.DataFrame(items)
columns = ['item_id', 'sales_type']

In [123]:
df

,section_type,item_id,images_thumbnail,sales_type,sales_title,deposit,rent,size_m2,공급면적,전용면적,...,is_zzim,status,service_type,tags,address1,address2,address3,manage_cost,reg_date,is_new
0,None,32819635,https://ic.zigbang.com/ic/items/32819635/1.jpg,전세,전세,16500,0,39.89,"{'m2': 39.89, 'p': '12.1'}","{'m2': 20.82, 'p': '6.3'}",...,False,True,오피스텔,[],서울시 서초구 서초동,None,None,7.5,2022-07-30T13:43:13+09:00,False
1,None,32838798,https://ic.zigbang.com/ic/items/32838798/1.jpg,월세,월세,1000,68,37.56,"{'m2': 37.56, 'p': '11.4'}","{'m2': 19.6, 'p': '5.9'}",...,False,True,오피스텔,[추천],서울시 서초구 서초동,None,None,7.5,2022-08-03T14:37:56+09:00,True
2,None,32872608,https://ic.zigbang.com/ic/items/32872608/1.jpg,월세,월세,1000,70,37.56,"{'m2': 37.56, 'p': '11.4'}","{'m2': 19.6, 'p': '5.9'}",...,False,True,오피스텔,[추천],서울시 서초구 서초동,None,None,7.5,2022-08-03T19:08:33+09:00,True
3,None,32838700,https://ic.zigbang.com/ic/items/32838700/1.jpg,전세,전세,50400,0,66.12,"{'m2': 66.12, 'p': '20'}","{'m2': 52.56, 'p': '15.9'}",...,False,True,빌라,[추천],서울시 서초구 서초동,None,None,9,2022-08-01T15:05:10+09:00,False
4,None,32767020,https://ic.zigbang.com/ic/items/32767020/1.jpg,월세,월세,20000,90,60.12,"{'m2': 60.12, 'p': '18.2'}","{'m2': 59.61, 'p': '18'}",...,False,True,빌라,[],서울시 서초구 서초동,None,None,5,2022-08-03T17:05:14+09:00,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,None,32850137,https://ic.zigbang.com/ic/items/32850137/1.jpg,월세,월세,300,260,32.91,"{'m2': 32.91, 'p': '10'}","{'m2': 27.22, 'p': '8.2'}",...,False,True,빌라,[추천],서울시 강남구 역삼동,None,None,10,2022-08-02T11:59:56+09:00,False
896,None,32852414,https://ic.zigbang.com/ic/items/32852414/1.jpg,월세,월세,130,250,33.44,"{'m2': 33.44, 'p': '10.1'}","{'m2': 27.65, 'p': '8.4'}",...,False,True,빌라,[추천],서울시 강남구 역삼동,None,None,10,2022-08-02T15:04:01+09:00,True
897,None,32809154,https://ic.zigbang.com/ic/items/32809154/1.jpg,월세,월세,120,70,19.83,"{'m2': 19.83, 'p': '6'}","{'m2': 19.83, 'p': '6'}",...,False,True,원룸,[추천],서울시 강남구 역삼동,None,None,9,2022-07-29T15:01:31+09:00,False
898,None,32844154,https://ic.zigbang.com/ic/items/32844154/1.jpg,월세,월세,100,100,29.75,"{'m2': 29.75, 'p': '9'}","{'m2': 29.75, 'p': '9'}",...,False,True,원룸,[추천],서울시 강남구 역삼동,None,None,9,2022-08-01T18:43:35+09:00,False


In [124]:
df[['item_id', 'sales_type']]

,item_id,sales_type
0,32819635,전세
1,32838798,월세
2,32872608,월세
3,32838700,전세
4,32767020,월세
...,...,...
895,32850137,월세
896,32852414,월세
897,32809154,월세
898,32844154,월세
